In [1]:
import pandas as pd
import numpy as np
import json
from scipy.spatial import KDTree

In [2]:
df = pd.read_csv('ep187_electrodes.csv')

In [3]:
x = []
y = []
z = []
positions = []

In [4]:
for index, row in df.iterrows():
    pos = json.loads(row.position)
    positions.append(pos)
    x.append(pos[0])
    y.append(pos[1])
    z.append(pos[2])

In [5]:
df['x'] = x
df['y'] = y
df['z'] = z

In [6]:
df2 = pd.read_csv('brain_new.csv')

In [7]:
df2 = df2.rename(columns={'Points:0': 'x', 'Points:1': 'y', 'Points:2': 'z' })
df2.head()

,Normals:0,Normals:1,Normals:2,x,y,z
0,0.098496,-0.357716,-0.928621,99.650002,88.247200,41.569401
1,-0.065635,-0.223418,-0.972511,98.919403,88.758202,41.424801
2,0.138644,-0.136587,-0.980878,99.701897,88.787903,41.447201
3,-0.234908,-0.157347,-0.959198,98.091301,89.319397,41.429001
4,-0.005564,-0.004576,-0.999974,98.757103,89.491699,41.338200


In [8]:
allPoints = []
for index, row in df2.iterrows():
    point = [row.x, row.y, row.z]
    allPoints.append(point)

In [9]:
NDIM = 3 # number of dimensions

# read points into array
a = np.array(allPoints)
# print(a)
# find 10 nearest points
tree = KDTree(a, leafsize=a.shape[0]+1)

newPoints = []
newX = []
newY = []
newZ = []

for position in positions:
    point = np.array(position)
    distances, ndx = tree.query([point], k=1)
    nearPoint = [a[ndx][0][0], a[ndx][0][1], a[ndx][0][2]]
    newPoints.append(nearPoint)
    newX.append(a[ndx][0][0])
    newY.append(a[ndx][0][1])
    newZ.append(a[ndx][0][2])

In [10]:
df['newX'] = newX
df['newY'] = newY
df['newZ'] = newZ
df['newPosition'] = newPoints

In [11]:
df.head()

,electrode_number,E_EEG,E_Brain,position,position_marker,x,y,z,newX,newY,newZ,newPosition
0,11,E11,PS1,"[138.115783691406,166.815368652344,110.1925659...",Y,138.115784,166.815369,110.192566,138.582001,164.065002,116.200996,"[138.5820007, 164.0650024, 116.2009964]"
1,12,E12,PS2,"[142.168121337891,181.290649414063,112.6060028...",Y,142.168121,181.290649,112.606003,136.712997,176.985992,121.097000,"[136.7129974, 176.9859924, 121.0970001]"
2,13,E13,PS3,"[141.510070800781,183.282073974609,121.5120697...",Y,141.510071,183.282074,121.512070,138.268997,179.369995,124.501000,"[138.2689972, 179.3699951, 124.5009995]"
3,19,E19,AT1,"[134.525161743164,137.630401611328,79.41387939...",Y,134.525162,137.630402,79.413879,134.684998,137.822998,78.492302,"[134.6849976, 137.822998, 78.49230194]"
4,20,E20,AT2,"[139.903259277344,145.456604003906,83.08189392...",Y,139.903259,145.456604,83.081894,140.056000,145.141007,84.123497,"[140.0559998, 145.1410065, 84.12349701]"


In [12]:
df.to_csv('ep187_electrodes_new.csv', index=False)